In [1]:
import numpy as np
import time 

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter

from qcodes.sweep import sweep, nest, chain, szip, getter, setter
from qcodes.sweep.sweep import BaseSweepObject, ParametersTable, wrap_objects, time_trace

from qcodes.sweep.base_sweep import (
    Nest, Zip, Chain, ParameterSweep, ParameterWrapper, FunctionSweep, FunctionWrapper, While
)

c:\users\a-sochat\development\qcodes\qcodes\utils\deferred_operations.py:45: UserWarning: DeferredOperations has been deprecated and will be removed in the not-too-distant future.
  warnings.warn("DeferredOperations has been deprecated and will be removed "


In [2]:
class Printer:
    def __init__(self, sweep_object): 
        self._ind, self._dep = sweep_object.parameter_table.flatten()
        self._symbols_list = sweep_object.parameter_table.symbols_list()
        self._inferred_symbols_list = sweep_object.parameter_table.inferred_symbols_list()
        
    def __enter__(self): 
        header_ind = (2*"\t").join(["{} [{}]".format(*i) for i in self._ind.items()])
        header_dep = (2*"\t").join(["{} [{}]".format(*i) for i in self._dep.items()])
        sep = " | "
        
        print(self._inferred_symbols_list)
        print()
        print((header_ind + sep + "\t" + header_dep).strip(sep))
        
        return self
    
    def __exit__(self, type, value, traceback): 
        pass

    def __call__(self, result):
        print(
            " " + "\t ".join([
                "{:.2e}".format(result[ip]) if result[ip] is not None else "None    " for ip in self._symbols_list
            ])
        )

In this notebook, we illustrate how pysweep incorporates the nation of inferred parameters. We will first illustrate inferring from independent parameters

In [3]:
x = ManualParameter("x", unit="V")
yparam = ManualParameter("y", unit="V")

# The "setter" decorator lets us use arbitrary functions to set sweep parameters. The value of any 
# declared inferred parameters should be returned by the function 
@setter([("y", "mV")], inferred_parameters=[("yv", "V")]) # We are given mV in loops 
def ysetter(y):
    yv = y / 1000
    yparam.set(yv)
    return yv

m = ManualParameter("m", unit="A")
m.get = lambda: x() ** 2

n = ManualParameter("n", unit="A")
n.get = lambda: x() - yparam() ** 2 + 16

In [4]:
sweep_object = sweep(x, [0, 1, 2])(
    m,
    sweep(ysetter, [0, 1000, 2000])(
        n  
    )
)

with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

yv inferred from y

x [V]		y [mV]		yv [V] | 	m [A]		n [A]
 0.00e+00	 None    	 None    	 0.00e+00	 None    
 0.00e+00	 0.00e+00	 0.00e+00	 None    	 1.60e+01
 0.00e+00	 1.00e+03	 1.00e+00	 None    	 1.50e+01
 0.00e+00	 2.00e+03	 2.00e+00	 None    	 1.20e+01
 1.00e+00	 None    	 None    	 1.00e+00	 None    
 1.00e+00	 0.00e+00	 0.00e+00	 None    	 1.70e+01
 1.00e+00	 1.00e+03	 1.00e+00	 None    	 1.60e+01
 1.00e+00	 2.00e+03	 2.00e+00	 None    	 1.30e+01
 2.00e+00	 None    	 None    	 4.00e+00	 None    
 2.00e+00	 0.00e+00	 0.00e+00	 None    	 1.80e+01
 2.00e+00	 1.00e+03	 1.00e+00	 None    	 1.70e+01
 2.00e+00	 2.00e+03	 2.00e+00	 None    	 1.40e+01


In [3]:
magnet_x = ManualParameter("mag_x", unit="T")
magnet_y = ManualParameter("mag_y", unit="T")
magnet_z = ManualParameter("mag_z", unit="T")

@setter(
    [("r", "T"), ("th", "r"), ("ph", "r")], 
    inferred_parameters=[
        ("mx", "T"), 
        ("my", "T"),
        ("mz", "T")
    ]
)
def mag_setter(r, theta, phi): 
    mag_z = r * np.cos(theta)
    mag_x = r * np.sin(theta) * np.cos(phi)
    mag_y = r * np.sin(theta) * np.sin(phi)
    
    magnet_x.set(mag_x)
    magnet_y.set(mag_y)
    magnet_z.set(mag_z)
    
    return mag_x, mag_y, mag_z

In [6]:
theta = 0.1
phi = -0.2
r = [0, 1, 2, 3, 4]

coordinates = zip(r, len(r) * [theta], len(r) * [phi])

In [7]:
sweep_object = sweep(mag_setter, coordinates)

In [8]:
with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

mx inferred from r,th,ph
my inferred from r,th,ph
mz inferred from r,th,ph

r [T]		th [r]		ph [r]		mx [T]		my [T]		mz [T] | 	
 0.00e+00	 1.00e-01	 -2.00e-01	 0.00e+00	 -0.00e+00	 0.00e+00
 1.00e+00	 1.00e-01	 -2.00e-01	 9.78e-02	 -1.98e-02	 9.95e-01
 2.00e+00	 1.00e-01	 -2.00e-01	 1.96e-01	 -3.97e-02	 1.99e+00
 3.00e+00	 1.00e-01	 -2.00e-01	 2.94e-01	 -5.95e-02	 2.99e+00
 4.00e+00	 1.00e-01	 -2.00e-01	 3.91e-01	 -7.93e-02	 3.98e+00


NB: In my opinion the driver for the magnet driver AMI430 is over complicated and should *not* include math code to convert spherical to cartesian coordinates at all. Proper sweeping code for the magnet is shown above 

In [9]:
@getter([("mag", "T")])
def mag_getter(): 
    mag_x = magnet_x.get()
    mag_y = magnet_y.get()
    mag_z = magnet_z.get()
    return np.sqrt(mag_x**2 + mag_y**2 + mag_z**2)

In [10]:
coordinates = zip(r, len(r) * [theta], len(r) * [phi])
sweep_object = sweep(mag_setter, coordinates)(mag_getter)

In [11]:
with Printer(sweep_object) as printer:
    for i in sweep_object:   
        printer(i)

mx inferred from r,th,ph
my inferred from r,th,ph
mz inferred from r,th,ph

r [T]		th [r]		ph [r]		mx [T]		my [T]		mz [T] | 	mag [T]
 0.00e+00	 1.00e-01	 -2.00e-01	 0.00e+00	 -0.00e+00	 0.00e+00	 0.00e+00
 1.00e+00	 1.00e-01	 -2.00e-01	 9.78e-02	 -1.98e-02	 9.95e-01	 1.00e+00
 2.00e+00	 1.00e-01	 -2.00e-01	 1.96e-01	 -3.97e-02	 1.99e+00	 2.00e+00
 3.00e+00	 1.00e-01	 -2.00e-01	 2.94e-01	 -5.95e-02	 2.99e+00	 3.00e+00
 4.00e+00	 1.00e-01	 -2.00e-01	 3.91e-01	 -7.93e-02	 3.98e+00	 4.00e+00


In [6]:
mag_setter.parameter_table

mx inferred from r,th,ph
my inferred from r,th,ph
mz inferred from r,th,ph
r [T],th [r],ph [r],mx [T],my [T],mz [T]|